In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import cv2
import matplotlib.image as mpimg
from PIL import Image, ImageDraw
from numpy import asarray
import re
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io
from tqdm import tqdm

ModuleNotFoundError: No module named 'cv2'

In [ ]:
train=pd.read_csv("train.csv")

In [ ]:
train2 = train[["image_id","bbox"]]

In [ ]:
train2['x'] = -1
train2['y'] = -1
train2['w'] = -1
train2['h'] = -1

In [ ]:
train2.head()

In [ ]:
def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

In [ ]:
train2[['x', 'y', 'w', 'h']] = np.stack(train2['bbox'].apply(lambda x: expand_bbox(x)))
train2.drop(columns=['bbox'], inplace=True)
train2['x'] = train2['x'].astype(np.float)
train2['y'] = train2['y'].astype(np.float)
train2['w'] = train2['w'].astype(np.float)
train2['h'] = train2['h'].astype(np.float)

In [ ]:
train2.head()

In [ ]:
train2["x_max"] = train2.apply(lambda col: col.x + col.w, axis=1)
train2["y_max"] = train2.apply(lambda col: col.y + col.h, axis = 1)

In [ ]:
train2.head()

In [ ]:
train3 = train2.drop(["w","h"],axis=1)

In [ ]:
train3.columns = ["image_id","x_min","y_min","x_max","y_max"]

In [ ]:
train3.head()

In [ ]:
train3["x_mid"] = train3.apply(lambda col : (col.x_min + col.x_max)/2,axis=1)
train3["y_mid"] = train3.apply(lambda col : (col.y_min + col.y_max)/2,axis=1)

In [ ]:
train3.head()

In [ ]:
train3.image_id.value_counts()

In [ ]:
img_df = train3[train3['image_id']=="b6ab77fd7"]

In [ ]:
img_df.head()

In [ ]:
train3.head()

In [ ]:
#train3[train3["image_id"]=="b6ab77fd7"]
path = "/kaggle/input/global-wheat-detection/train/"
# io.imread(path + "b6ab77fd7.jpg" )

In [ ]:
def plot_bbox(img_id):
    img_df = train3[train3['image_id']==img_id]
    img = io.imread(path + img_id + ".jpg")
    for index, row in img_df.iterrows():
        xmin = row['x_min']
        ymin = row['y_min']
        xmax = row['x_max']
        ymax = row['y_max']
#         label_name = row['source']
#         class_series = train[train["source"]==label_name]
#         class_name = class_series["source"].values[0]
#         print(f"Coordinates: {xmin,ymin}, {xmax,ymax}")
        cv2.rectangle(img, (int(xmin),int(ymin)), (int(xmax),int(ymax)), (255,0,0), 2)
        #font = cv2.FONT_HERSHEY_SIMPLEX
        #cv2.putText(img, class_name, (xmin,ymin-10), font, 1, (0,255,0), 5)
    plt.figure(figsize=(15,10))
    plt.title('Image with Bounding Box')
    plt.imshow(img)
    plt.axis("off")
    plt.show()

In [ ]:
plot_bbox("35b935b6c")

In [ ]:
#create anchor boxes:

In [ ]:
boxes = np.array(train2[["w","h"]].values.tolist())

In [ ]:
class YOLO_Kmeans:

    def __init__(self, cluster_number, boxes):
        self.cluster_number = cluster_number
        #self.filename = "2012_train.txt"

    def iou(self, boxes, clusters):  # 1 box -> k clusters
        n = boxes.shape[0]
        k = self.cluster_number

        box_area = boxes[:, 0] * boxes[:, 1]
        box_area = box_area.repeat(k)
        box_area = np.reshape(box_area, (n, k))

        cluster_area = clusters[:, 0] * clusters[:, 1]
        cluster_area = np.tile(cluster_area, [1, n])
        cluster_area = np.reshape(cluster_area, (n, k))

        box_w_matrix = np.reshape(boxes[:, 0].repeat(k), (n, k))
        cluster_w_matrix = np.reshape(np.tile(clusters[:, 0], (1, n)), (n, k))
        min_w_matrix = np.minimum(cluster_w_matrix, box_w_matrix)

        box_h_matrix = np.reshape(boxes[:, 1].repeat(k), (n, k))
        cluster_h_matrix = np.reshape(np.tile(clusters[:, 1], (1, n)), (n, k))
        min_h_matrix = np.minimum(cluster_h_matrix, box_h_matrix)
        inter_area = np.multiply(min_w_matrix, min_h_matrix)

        result = inter_area / (box_area + cluster_area - inter_area)
        return result

    def avg_iou(self, boxes, clusters):
        accuracy = np.mean([np.max(self.iou(boxes, clusters), axis=1)])
        return accuracy

    def kmeans(self, boxes, k, dist=np.median):
        box_number = boxes.shape[0]
        distances = np.empty((box_number, k))
        last_nearest = np.zeros((box_number,))
        np.random.seed()
        clusters = boxes[np.random.choice(
            box_number, k, replace=False)]  # init k clusters
        while True:

            distances = 1 - self.iou(boxes, clusters)

            current_nearest = np.argmin(distances, axis=1)
            if (last_nearest == current_nearest).all():
                break  # clusters won't change
            for cluster in range(k):
                clusters[cluster] = dist(  # update clusters
                    boxes[current_nearest == cluster], axis=0)

            last_nearest = current_nearest

        return clusters

#     def result2txt(self, data):
#         f = open("yolo_anchors.txt", 'w')
#         row = np.shape(data)[0]
#         for i in range(row):
#             if i == 0:
#                 x_y = "%d,%d" % (data[i][0], data[i][1])
#             else:
#                 x_y = ", %d,%d" % (data[i][0], data[i][1])
#             f.write(x_y)
#         f.close()

#     def txt2boxes(self):
#         f = open(self.filename, 'r')
#         dataSet = []
#         for line in f:
#             infos = line.split(" ")
#             length = len(infos)
#             for i in range(1, length):
#                 width = int(infos[i].split(",")[2]) - \
#                     int(infos[i].split(",")[0])
#                 height = int(infos[i].split(",")[3]) - \
#                     int(infos[i].split(",")[1])
#                 dataSet.append([width, height])
#         result = np.array(dataSet)
#         f.close()
#         return result

    def txt2clusters(self):
        #all_boxes = self.txt2boxes()
        result = self.kmeans(boxes, k=self.cluster_number)
        result = result[np.lexsort(result.T[0, None])]
        #self.result2txt(result)
        print("K anchors:\n {}".format(result))
        print("Accuracy: {:.2f}%".format(
            self.avg_iou(boxes, result) * 100))
        return result

In [ ]:
if __name__ == "__main__":
    cluster_number = 10
    #filename = "2012_train.txt"
    kmeans = YOLO_Kmeans(cluster_number,boxes)
    kmeans.txt2clusters()

In [ ]:
anchors=kmeans.txt2clusters()

In [ ]:
anchors

In [ ]:
# now the anchors are created.we need to create the data suitable for yolo model.

In [ ]:
train2.head()

In [ ]:
train2["image_id"] = train2["image_id"].apply(lambda x : x + ".jpg")

In [ ]:
train2.head()

In [ ]:
train2_filter = train2.drop(["x_max","y_max"],axis=1)

In [ ]:
train2_filter.head()

In [ ]:
train2_filter.insert(1, "confidence", 1.0)

In [ ]:
train2_filter.head()

In [ ]:
counts = train2_filter.image_id.value_counts().values.tolist()

In [ ]:
#train2_filter[["confidence",x","y","w","h"]].values.tolist()

In [ ]:
unique_images=train2_filter.image_id.unique().tolist()[0:1000]

In [ ]:
#train2_filter[train2_filter["image_id"]=="b6ab77fd7.jpg"][["x","y","w","h"]].values.tolist()

In [ ]:
values=[]
zeros = np.zeros((116,5))
for img in tqdm(unique_images):
    value = train2_filter[train2_filter["image_id"]==img][["confidence","x","y","w","h"]].values.tolist()
    zeros[:len(value)] = np.array(value)
    values.append(zeros.tolist())

In [ ]:
#preprocessing true boxes:

In [ ]:
#y_true[0,0,0,0,0:5] = np.array(values)[0,0,0:5]  #works

In [ ]:
input_shape = (1024,1024)

In [ ]:
# def preprocess_true_boxes(true_boxes, input_shape, anchors):

#     #assert (true_boxes[..., 4]<num_classes).all(), 'class id must be less than num_classes'
#     #num_layers = len(anchors)//3 # default setting
#     num_layers = 3
#     anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] #if num_layers==3 else [[3,4,5], [1,2,3]]

#     true_boxes = np.array(true_boxes, dtype='float32')
#     input_shape = np.array(input_shape, dtype='int32')
#     boxes_xy = ((true_boxes[..., 1:3] + true_boxes[...,3:5]) + (true_boxes[...,1:3]))//2
#     boxes_wh = true_boxes[...,3:5]
#     true_boxes[..., 1:3] = boxes_xy
#     #true_boxes[..., 2:4] = boxes_wh/input_shape[::-1]

#     m = true_boxes.shape[0]
#     #grid_shapes = [input_shape//{0:32, 1:16, 2:8}[l] for l in range(num_layers)]
#     y_true = np.zeros((m,64,64,3,5),dtype='float32')

#     # Expand dim to apply broadcasting.
#     anchors = np.expand_dims(anchors, 0)
#     anchor_maxes = anchors / 2.
#     anchor_mins = -anchor_maxes
#     valid_mask = boxes_wh[..., 0]>0

#     for b in range(m):
#         # Discard zero rows.
#         wh = boxes_wh[b, valid_mask[b]]
#         if len(wh)==0: continue
#         # Expand dim to apply broadcasting.
#         wh = np.expand_dims(wh, -2)
#         box_maxes = wh / 2.
#         box_mins = -box_maxes

#         intersect_mins = np.maximum(box_mins, anchor_mins)
#         intersect_maxes = np.minimum(box_maxes, anchor_maxes)
#         intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
#         intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
#         box_area = wh[..., 0] * wh[..., 1]
#         anchor_area = anchors[..., 0] * anchors[..., 1]
#         iou = intersect_area / (box_area + anchor_area - intersect_area)

#         # Find best anchor for each true box
#         best_anchor = np.argmax(iou, axis=-1)

#         for t, n in enumerate(best_anchor):
#             for l in range(num_layers):
#                 if n in anchor_mask[l]:
#                     i = np.floor(true_boxes[b,t,1]/16).astype('int32')
#                     j = np.floor(true_boxes[b,t,2]/16).astype('int32')
#                     #k = anchor_mask[l].index(n)
#                     #c = true_boxes[b,t, 4].astype('int32')
#                     y_true[b, j, i,1:5] = true_boxes[b,t, 1:5]
#                     y_true[b, j, i,0] = 1
#                     #y_true[l][b, j, i, k, 5+c] = 1

#     return y_true

In [ ]:
tru = np.array(values)

In [ ]:
y_true = np.zeros((1000,64,64,5),dtype='float32')

In [ ]:
i = np.floor(tru[...,1:2]/16).astype('int32')
j = np.floor(tru[...,2:3]/16).astype('int32')
for r in range(1000):
    i_ = i[r,...,0]
    j_ = j[r,...,0]
    y_true[r,i_,j_,1:5] = tru[r,...,1:5]
    y_true[r,i_,j_,0] = 1

In [ ]:
# the above thing works, but check the algorithm again

In [ ]:
#now converting the image into appropriate format.

In [ ]:
#io.imread(path+unique_images[0])

In [ ]:
def datagenerator(images,labels,batchsize, mode="train"):
    while True:
        start = 0
        end = batchsize
        while start  < len(unique_images): 
            x = unique_images[start:end] 
            images_list=[]
            for i in x:
                images_list.append(io.imread(path+i))
            y = labels[start:end]
            yield np.array(images_list),np.array(y)

            start += batchsize
            end += batchsize

In [ ]:
dataset=datagenerator(unique_images,y_true,10)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# import sys
# local_vars = list(locals().items())
# for var, obj in local_vars:
#     print(var, sys.getsizeof(obj))

In [ ]:
#creating a small model:(just to check the output):

In [ ]:
import keras
print('keras: %s' % keras.__version__)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
#activation=lambda l : tf.concat([tf.sigmoid(l[...,0:1]),l[...,1:5]], axis=-1),

In [ ]:
def trail1():
    inputs = tf.keras.Input(shape=(1024,1024, 3), name='img')
    input2 = tf.keras.layers.Conv2D(128,3,strides=(2,2),padding="same",activation="relu",name="just_for_fun")(inputs)
    input3 = tf.keras.layers.Conv2D(64,3,strides=(2,2),padding="same",activation="relu",name="just_for_fun1")(input2)
    input4 = tf.keras.layers.Conv2D(32,3,strides=(2,2),padding="same",activation="relu",name="just_for_fun2")(input3)
    input5 = tf.keras.layers.Conv2D(16,3,strides=(2,2),padding="same",activation="relu",name="just_for_fun3")(input4)
    input6 = tf.keras.layers.Conv2D(5,1,activation=lambda l : tf.concat([tf.sigmoid(l[...,0:1]),l[...,1:5]], axis=-1),name="just_for_fun4")(input5)
    #input6 = tf.keras.layers.Flatten()(input5)
    #outputs = tf.keras.layers.Dense(580)(input6)
    model = tf.keras.Model(inputs, input6, name='trail1')
    return model

In [ ]:
tt=trail1()

In [ ]:
tt.summary()

In [ ]:
#compiling the model

In [ ]:
#yolo loss:

In [ ]:
def loss(y_true, y_pred):
    obj_loss = tf.keras.losses.binary_crossentropy(y_true[...,0:1], y_pred[...,0:1])
    box_loss = tf.reduce_sum(tf.math.squared_difference(y_true[...,1:5], y_pred[...,1:5]))
    return tf.reduce_sum(obj_loss) + box_loss

In [ ]:
tt.compile(optimizer='adam',
              loss=loss,
              metrics=['acc'])

In [ ]:
history=tt.fit(dataset,steps_per_epoch = 100, epochs=1)

In [ ]:
#test data

In [ ]:
test = os.listdir("/kaggle/input/global-wheat-detection/test")

In [ ]:
test_path = "/kaggle/input/global-wheat-detection/test/"

In [ ]:
#Image.open(test_path + test[0])

In [ ]:
test_img = []
for img in test:
    test_img.append(io.imread(test_path+img))

In [ ]:
test_history = tt.predict(np.array(test_img))

In [ ]:
test_history[0,...,0][0]

In [ ]:
#worst model ever LOL::;